In [1]:
import json

import pandas as pd
import plotly.graph_objects as go


###################################


from datafin.apis import PolygonClient                     #type: ignore

from datafin.aws import SecretsClient                      #type: ignore
from datafin.aws import S3Client                           #type: ignore

from datafin.utils import GmailClient                      #type: ignore
from datafin.utils import (                                #type: ignore
    now,
    to_ny_time,
    string_formating,
    format_date,
    get_ny_timestamp_for_today_time_range,
    is_today_a_trading_day
)

In [2]:
secrets = SecretsClient()

s3 = S3Client(
    aws_access_key_id = secrets.aws_access_key,
    aws_secret_access_key = secrets.aws_secret_access_key,
    bucket_name = secrets.get_bucket_name()
)

In [ ]:

def portfolio_list(
) -> list:
    
    s3 = S3Client(
        bucket_name = secrets.get_bucket_name(),
        aws_access_key_id = secrets.aws_access_key,
        aws_secret_access_key = secrets.aws_secret_access_key,
        region_name = 'us-east-1'
    )

    raw_json_list = s3.get_json(
        path = 'v1/reference',
        file_name = 'portfolio'
    )

    cleaned_list = list(raw_json_list['data'])
    return cleaned_list


def portfolio_combined_df(
        input_list
) -> pd.DataFrame:
    
    pg = PolygonClient(
        api_key = secrets.get_polygon_api_key()
    )

    timestamps = get_ny_timestamp_for_today_time_range(
        _from = (9, 30, 0),
        _to = (16, 0, 0)
    )

    df_start = pd.DataFrame()

    for symbol in input_list:
        aggs = pg.get_aggs(
            symbol = symbol,
            multiplier = 5,
            unit = 'minute',
            _from = timestamps[0],
            _to = timestamps[1]
        )
        df = pd.DataFrame(aggs)
        df['symbol'] = symbol
        df_start = pd.concat([df_start, df], ignore_index=True)
    return df_start

def portfolio_with_previous_close_df(
        input_df: pd.DataFrame,
) -> pd.DataFrame:

    pg = PolygonClient(
            api_key = secrets.get_polygon_api_key()
        )

    symbols_from_df = input_df['symbol'].unique()

    previous_closes = {}

    for symbol in symbols_from_df:
        previous_close = pg.get_previous_close_agg(symbol)[0].close
        previous_closes[symbol] = previous_close

    input_df['previous_close'] = input_df['symbol'].map(previous_closes)

    return input_df

def portfolio_cleaned_df(
        input_df: pd.DataFrame
) -> pd.DataFrame:
    
    input_df = input_df.drop(columns=['otc'])
    input_df['datetime_ny'] = pd.to_datetime(input_df['timestamp'], unit='ms', utc=True).dt.tz_convert('America/New_York')
    
    input_df['perf_from_prev_close_nom'] = input_df['close'] - input_df['previous_close']
    input_df['perf_from_prev_close_per'] = ((input_df['close'] - input_df['previous_close']) / input_df['previous_close']) * 100
    input_df = input_df.reset_index(drop=True)
    
    return input_df

def portfolio_data_prep_df(
        input_df: pd.DataFrame
) -> pd.DataFrame:
    
    sorted_df = input_df.sort_values(['symbol', 'datetime_ny'])
    latest_closes_per_symbol = sorted_df.groupby('symbol').last().reset_index()

    perfomance_df_for_share = latest_closes_per_symbol[['symbol', 'close', 'perf_from_prev_close_nom', 'perf_from_prev_close_per']]

    return perfomance_df_for_share




list_ = portfolio_list()
df_init = portfolio_combined_df(list_)
df_with_close = portfolio_with_previous_close_df(df_init)
df = portfolio_cleaned_df(df_with_close)
df_for_data_summary = portfolio_data_prep_df(df)
df.head(5)

,open,high,low,close,volume,vwap,timestamp,transactions,symbol,previous_close,datetime_ny,perf_from_prev_close_nom,perf_from_prev_close_per
0,215.0900,216.760,215.020,215.2917,1310821.0,215.7501,1750253400000,16256,AMZN,214.82,2025-06-18 09:30:00-04:00,0.4717,0.219579
1,215.3100,215.850,215.150,215.4998,571384.0,215.4817,1750253700000,8804,AMZN,214.82,2025-06-18 09:35:00-04:00,0.6798,0.316451
2,215.4800,215.880,215.400,215.4050,461784.0,215.6012,1750254000000,7059,AMZN,214.82,2025-06-18 09:40:00-04:00,0.5850,0.272321
3,215.4100,215.870,215.260,215.8012,282203.0,215.5140,1750254300000,5373,AMZN,214.82,2025-06-18 09:45:00-04:00,0.9812,0.456754
4,215.8000,216.450,215.800,216.1100,561028.0,216.1660,1750254600000,8646,AMZN,214.82,2025-06-18 09:50:00-04:00,1.2900,0.600503
...,...,...,...,...,...,...,...,...,...,...,...,...,...
548,296.0600,298.000,296.015,297.5800,781011.0,297.1780,1750275600000,12798,COIN,253.85,2025-06-18 15:40:00-04:00,43.7300,17.226709
549,297.5632,299.320,296.800,296.9900,916822.0,298.3351,1750275900000,14307,COIN,253.85,2025-06-18 15:45:00-04:00,43.1400,16.994288
550,297.0000,297.308,294.350,294.8000,1066749.0,295.4052,1750276200000,16363,COIN,253.85,2025-06-18 15:50:00-04:00,40.9500,16.131574
551,294.8400,295.880,294.170,295.2400,1305194.0,295.1348,1750276500000,20840,COIN,253.85,2025-06-18 15:55:00-04:00,41.3900,16.304904


In [8]:
import plotly.graph_objects as go
from IPython.display import Image, display

##########################

# df = s3.get_parquet(
#     path = 'v1/data/test',
#     file_name = 'spy-open-chart-test'
# )


def generate_chart(
        df,
        title=None
):
        
    fig = go.Figure(data=[
            go.Candlestick(
                x=df['datetime_ny'],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close']
            )
        ]
    )
    
    previous_close_value = df['previous_close'].iloc[0]

    fig.add_hline(
    y=previous_close_value,
    line_color="blue",
    line_width=1
    )


    df = df.reset_index(drop=True)
    date = format_date(df['datetime_ny'].head(1)[0].date())
    if title:
        fig.update_layout(
        title={
            'text': f'{title} | {date}',
            'x': 0.5,
            'xanchor': 'center',
            'font': {'size': 15,
                    'color': 'black'
            }
        }
)

    fig.update_layout(xaxis_rangeslider_visible=False)
        
    image_bytes = fig.to_image(
        format="png",
        scale=5,
        engine="kaleido"
    )

    return image_bytes

def portfolio_text_for_email(
        input_df: pd.DataFrame    
) -> str:
    
    performance_lines = []
    
    for _, row in input_df.iterrows():
        symbol = row['symbol']
        price = row['close']
        pct_change = row['perf_from_prev_close_per']
        nom_change = row['perf_from_prev_close_nom']
        
        pct_sign = "+" if pct_change >= 0 else ""
        pct_formatted = f"{pct_sign}{pct_change:.2f}%"
        
        if nom_change >= 0:
            nom_formatted = f"+${nom_change:.2f}"
        else:
            nom_formatted = f"-${abs(nom_change):.2f}"
        
        line = f"{symbol:<6} ........ {price:>8.2f} ........ {pct_formatted:>7} ........ {nom_formatted:>8}"
        performance_lines.append(line)
    
    return_string = "\n".join(performance_lines)
    return return_string

symbols = df['symbol'].unique()

chart_dict = {}
for i in symbols:

    symbol_df = df[df['symbol'] == i]
    chart = generate_chart(symbol_df, title=i)

    chart_dict[i] = chart


email_text = portfolio_text_for_email(
    df_for_data_summary
)

In [9]:
gmail = GmailClient(
    sender_email = secrets.get_gmail_address(),
    app_password = secrets.get_gmail_app_password()
)

gmail.send_email(
    to = secrets.get_gmail_send_to_address(),
    subject = "today's portfolio performance",
    text = email_text,
    png_dict = chart_dict
)